In [1]:
from bswrangler import *

In [2]:
 df = query_database('NYY_NYM_2020', '''
                        SELECT player_name, pitcher, game_date, game_type, home_team,
                        away_team, pitch_type, pitch_name, release_speed,
                        release_spin_rate, events, type, description, zone,
                        release_pos_x, release_pos_z, pfx_x, pfx_z, plate_x, plate_z,
                        release_extension, vx0, vy0, vz0, ax, ay, az
                        FROM statcast
                        WHERE pitch_type IS NOT NULL; ''')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17566 entries, 0 to 17565
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   player_name        17566 non-null  object 
 1   pitcher            17566 non-null  int64  
 2   game_date          17566 non-null  object 
 3   game_type          17566 non-null  object 
 4   home_team          17566 non-null  object 
 5   away_team          17566 non-null  object 
 6   pitch_type         17566 non-null  object 
 7   pitch_name         17566 non-null  object 
 8   release_speed      17566 non-null  float64
 9   release_spin_rate  17565 non-null  float64
 10  events             4391 non-null   object 
 11  type               17566 non-null  object 
 12  description        17566 non-null  object 
 13  zone               17566 non-null  int64  
 14  release_pos_x      17566 non-null  float64
 15  release_pos_z      17566 non-null  float64
 16  pfx_x              175

In [4]:
make_datetime(df)

In [5]:
df[df.pitch_type == 'CS']

,player_name,pitcher,game_date,game_type,home_team,away_team,pitch_type,pitch_name,release_speed,release_spin_rate,...,pfx_z,plate_x,plate_z,release_extension,vx0,vy0,vz0,ax,ay,az
2000,Michael King,650633,2020-08-30,R,NYY,NYM,CS,Curveball,56.7,1108.0,...,1.5,0.22,6.37,7.0,2.605929,-81.977472,8.225544,-0.508284,11.843688,-26.880928


In [6]:
rename_to_curveball(df, [2000])

In [7]:
change_values(df, 'events', ['caught_stealing_2b', 'caught_stealing_3b',
                             'pickoff_caught_stealing_2b', 'run', 'pickoff_2b',
                             'other_out', 'strikeout_double_play', 'interf_def',
                             'fielders_choice_out', 'fielders_choice',
                             'grounded_into_double_play', 'force_out', 'double_play',
                             'sac_bunt', 'sac_fly', 'hit_by_pitch'],
              [None, None, None, None, None, None, 'strikeout', 'field_error',
               'field_out', 'field_out', 'field_out', 'field_out', 'field_out',
               'field_out', 'field_out', 'walk'])

change_values(df, 'description', ['foul_bunt', 'swinging_strike_blocked',
                                  'missed_bunt','foul_tip', 'foul', 'blocked_ball',
                                  'pitchout'], 
              ['swinging_strike', 'swinging_strike', 'swinging_strike', 'swinging_strike',
               'swinging_strike','ball', 'ball'])

In [8]:
df.description.unique()

array(['swinging_strike', 'ball', 'hit_into_play', 'called_strike',
       'hit_into_play_no_out', 'hit_into_play_score', 'hit_by_pitch'],
      dtype=object)

In [9]:
df.events.unique()

array(['strikeout', None, 'field_out', 'single', 'home_run', 'walk',
       'double', 'field_error', 'triple'], dtype=object)

In [10]:
backwards_k(df)

/Users/stevennewton/Documents/GitHub/MLB_Statcast/src/bswrangler.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["events"][ix] = 'called_' + df["events"][ix]


In [11]:
backfiller(df, "events", "description")

/Users/stevennewton/Documents/GitHub/MLB_Statcast/src/bswrangler.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_to_fill][ix] = df[col_to_fill_from][ix]


In [12]:
df.events[:20]

0           strikeout
1                ball
2     swinging_strike
3     swinging_strike
4           field_out
5       called_strike
6           field_out
7           field_out
8     swinging_strike
9     swinging_strike
10               ball
11      called_strike
12          field_out
13               ball
14               ball
15             single
16               ball
17    swinging_strike
18    swinging_strike
19               ball
Name: events, dtype: object

In [13]:
df = onehot_encode(df, 'pitch_type')

In [14]:
df = onehot_encode(df, 'events')

In [15]:
df = drop_columns(df, ['description', 'pitch_type', 'pitch_name'])

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17566 entries, 0 to 17565
Data columns (total 45 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   player_name        17566 non-null  object        
 1   pitcher            17566 non-null  int64         
 2   game_date          17566 non-null  datetime64[ns]
 3   game_type          17566 non-null  object        
 4   home_team          17566 non-null  object        
 5   away_team          17566 non-null  object        
 6   release_speed      17566 non-null  float64       
 7   release_spin_rate  17565 non-null  float64       
 8   events             17566 non-null  object        
 9   type               17566 non-null  object        
 10  zone               17566 non-null  int64         
 11  release_pos_x      17566 non-null  float64       
 12  release_pos_z      17566 non-null  float64       
 13  pfx_x              17566 non-null  float64       
 14  pfx_z 

In [17]:
save_df(df, "NYY_NYM_2020")

TypeError: argument of type 'method' is not iterable